Note:
1. The file aggregatedData.csv is the clean dataset which aggregates all blog, magazine, and wiki data.
2. In the final dataframe, there are four columns related to score: author score, data source score, like score, and comment score.
3. I did not simulate the search process. But if the words input by the user matche with the words in the most_common_words column, then one match is one score.

In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
df = pd.read_csv("aggregatedData.csv",encoding = "ISO-8859-1")

In [3]:
df.head(5)

,Unnamed: 0,Article,Author,Comment,Content,Data_source,Date,Like,Most_common_words,Table of Contents,Title
0,0,NaN,Rick Beach,0.0,"by RICK BEACH, COPA?? SAFETY CHAIR\nThis unusu...",blog,12/30/2018 23:52,4.0,"['caps', 'pilot', 'accident', 'rocket', 'deplo...",How did the accident unfold?What evidence was ...,"CAPS Deployment Anomaly at Colorado Springs, C..."
1,1,NaN,Rick Beach,8.0,Every Cirrus aircraft built has a parachute sy...,blog,1/17/2016 0:50,0.0,"['caps', 'low', 'altitude', 'pull', 'attitude'...",How Low?How High?Know your Aircraft: The G5 is...,CAPS: How Low Can You Go? Or How High Do You N...
2,2,NaN,Rick Beach,2.0,(This article appears as the safety column in ...,blog,9/13/2017 23:27,1.0,"['cirrus', 'training', 'new', 'transition', 't...",Reach the Unreachable Low time-in-type vs low ...,"""Reach the Unreachable"" safety column in COPA ..."
3,3,NaN,Rick Beach,0.0,This column will appear in the Nov/Dec 2018 is...,blog,10/15/2018 4:22,12.0,"['runway', 'figure', 'left', 'airplane', 'wing...",NaN,?Â?That Sounds Expensive?Â? -- It Happened t...
4,4,NaN,Rick Beach,4.0,A Note from the Guest Editor\nThis June 2017 i...,blog,6/10/2017 18:11,0.0,"['copa', 'training', 'cirrus', 'safety', 'pilo...",A Note from the Guest Editor Success has Many ...,Guest Editor column to Special Safety Issue of...


Build table of Author: By the total number of likes he/she receved, number of work they published, time since the author joined the copa. However, we do not have the demographic data of user, so here time_with_copa is 0 for all.

In [4]:
author_table = df[['Author','Like']]
author_table_processed = author_table
author_table_processed.head(5)

,Author,Like
0,Rick Beach,4.0
1,Rick Beach,0.0
2,Rick Beach,1.0
3,Rick Beach,12.0
4,Rick Beach,0.0


In [5]:
author_list = list(df['Author'].dropna())
author_list

['Rick Beach',
 'Rick Beach',
 'Rick Beach',
 'Rick Beach',
 'Rick Beach',
 'Rick Beach',
 'Rick Beach',
 'Rick Beach',
 'Rick Beach',
 'Rick Beach',
 'Rick Beach',
 'Rick Beach',
 'Rick Beach',
 'Rick Beach',
 'Rick Beach',
 'Rick Beach',
 'Rick Beach',
 'Rick Beach',
 'Rick Beach',
 'Rick Beach',
 'Rick Beach',
 'Rick Beach',
 'Rick Beach',
 'Rick Beach',
 'Rick Beach',
 'Rick Beach',
 'Rick Beach',
 'Rick Beach',
 'Rick Beach',
 'Rick Beach',
 'GIL WILLIAMSON',
 'KIM BLONIGEN',
 'RICK BEACH',
 'RICK BEACH',
 'JERROLD SECKLER',
 'TRIP TAYLOR',
 'GIL WILLIAMSON',
 'KIM BLONIGEN',
 'RICK BEACH',
 'RICK BEACH',
 'JERROLD SECKLER',
 'TRIP TAYLOR',
 'GIL WILLIAMSON',
 'KIM BLONIGEN',
 'RICK BEACH',
 'RICK BEACH',
 'JERROLD SECKLER',
 'TRIP TAYLOR',
 'GIL WILLIAMSON',
 'KIM BLONIGEN',
 'RICK BEACH',
 'RICK BEACH',
 'JERROLD SECKLER',
 'TRIP TAYLOR',
 'GIL WILLIAMSON',
 'KIM BLONIGEN',
 'RICK BEACH',
 'RICK BEACH',
 'JERROLD SECKLER',
 'TRIP TAYLOR',
 'GIL WILLIAMSON',
 'KIM BLONIGEN',
 'RI

In [6]:
author_table_processed = author_table_processed[author_table_processed['Author'].isin(author_list)]
author_table_processed

,Author,Like
0,Rick Beach,4.0
1,Rick Beach,0.0
2,Rick Beach,1.0
3,Rick Beach,12.0
4,Rick Beach,0.0
5,Rick Beach,0.0
6,Rick Beach,4.0
7,Rick Beach,1.0
8,Rick Beach,0.0
9,Rick Beach,3.0


In [7]:
#For the same author, there are different representations of his/her name in different source. So transfer them all to lower case.
for i in range(len(author_table)):
    if (type(author_table['Author'][i])==str):
        author_table_processed['Author'][i] = author_table['Author'][i].lower()
author_table_processed.head(5)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\user\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


,Author,Like
0,rick beach,4.0
1,rick beach,0.0
2,rick beach,1.0
3,rick beach,12.0
4,rick beach,0.0


In [8]:
A = author_table_processed['Author'].unique()
AA = {'Author':list(A)}
AA = pd.DataFrame(AA)
A = list(AA['Author'].dropna())
len(A) #Except NaN, there are 27 authors in total.

27

In [9]:
author = pd.DataFrame({'Author':A})
author['num_of_work'] = 0
author['num_of_like'] = 0
author['time_with_copa'] = 0

In [10]:
#Count the number of work for each author.
for i in range(len(A)):
    author['num_of_work'][i] = list(author_table_processed['Author']).count(A[i])

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
#Count the number of like for each author.
for i in range(len(A)):
    author1 = author_table_processed.loc[author_table_processed['Author']==A[i]]
    author1 = author1.dropna()
    total_like = sum(author1['Like'])
    author['num_of_like'][i] = sum(author1['Like'])

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [12]:
author = author.fillna(0) #Since all columns are scores, so fill the NaN with 0.

In [13]:
total_num_of_work = sum(author['num_of_work'])
total_num_of_like = sum(author['num_of_like'])

In [14]:
author

,Author,num_of_work,num_of_like,time_with_copa
0,rick beach,90,37,0
1,gil williamson,30,0,0
2,kim blonigen,30,0,0
3,jerrold seckler,30,0,0
4,trip taylor,30,0,0
5,ctrease,1,0,0
6,mark craig,1,0,0
7,stuart simek,1,0,0
8,michael samp,1,0,0
9,john shaeffer,1,0,0


In [15]:
#Calculate score for number of work
score_num_of_work =list()
for i in range(len(A)):
    score_num_of_work.append(author['num_of_work'][i]/total_num_of_work)
author = pd.concat([author, pd.DataFrame(score_num_of_work)], axis=1)
author.columns =['Author','num_of_work','num_of_like','time_with_copa','score_num_of_work']
author

,Author,num_of_work,num_of_like,time_with_copa,score_num_of_work
0,rick beach,90,37,0,0.170778
1,gil williamson,30,0,0,0.056926
2,kim blonigen,30,0,0,0.056926
3,jerrold seckler,30,0,0,0.056926
4,trip taylor,30,0,0,0.056926
5,ctrease,1,0,0,0.001898
6,mark craig,1,0,0,0.001898
7,stuart simek,1,0,0,0.001898
8,michael samp,1,0,0,0.001898
9,john shaeffer,1,0,0,0.001898


In [16]:
#Calculate score for number of like
score_num_of_like =list()
for i in range(len(A)):
    score_num_of_like.append(author['num_of_like'][i]/total_num_of_like)
author = pd.concat([author, pd.DataFrame(score_num_of_like)], axis=1)
author.columns =['Author','num_of_work','num_of_like','time_with_copa','score_num_of_work','score_num_of_like']
author

,Author,num_of_work,num_of_like,time_with_copa,score_num_of_work,score_num_of_like
0,rick beach,90,37,0,0.170778,1.0
1,gil williamson,30,0,0,0.056926,0.0
2,kim blonigen,30,0,0,0.056926,0.0
3,jerrold seckler,30,0,0,0.056926,0.0
4,trip taylor,30,0,0,0.056926,0.0
5,ctrease,1,0,0,0.001898,0.0
6,mark craig,1,0,0,0.001898,0.0
7,stuart simek,1,0,0,0.001898,0.0
8,michael samp,1,0,0,0.001898,0.0
9,john shaeffer,1,0,0,0.001898,0.0


In [17]:
#Calculate total score for each author. Put more weight on num_of_like and num_of_work.
score = [score_num_of_work[i]*2 + score_num_of_like[i]*3 for i in range(len(score_num_of_work))] 
author = pd.concat([author, pd.DataFrame(score)], axis=1)
author.columns =['Author','num_of_work','num_of_like','time_with_copa','score_num_of_work','score_num_of_like','score']
author

,Author,num_of_work,num_of_like,time_with_copa,score_num_of_work,score_num_of_like,score
0,rick beach,90,37,0,0.170778,1.0,3.341556
1,gil williamson,30,0,0,0.056926,0.0,0.113852
2,kim blonigen,30,0,0,0.056926,0.0,0.113852
3,jerrold seckler,30,0,0,0.056926,0.0,0.113852
4,trip taylor,30,0,0,0.056926,0.0,0.113852
5,ctrease,1,0,0,0.001898,0.0,0.003795
6,mark craig,1,0,0,0.001898,0.0,0.003795
7,stuart simek,1,0,0,0.001898,0.0,0.003795
8,michael samp,1,0,0,0.001898,0.0,0.003795
9,john shaeffer,1,0,0,0.001898,0.0,0.003795


In [18]:
author

,Author,num_of_work,num_of_like,time_with_copa,score_num_of_work,score_num_of_like,score
0,rick beach,90,37,0,0.170778,1.0,3.341556
1,gil williamson,30,0,0,0.056926,0.0,0.113852
2,kim blonigen,30,0,0,0.056926,0.0,0.113852
3,jerrold seckler,30,0,0,0.056926,0.0,0.113852
4,trip taylor,30,0,0,0.056926,0.0,0.113852
5,ctrease,1,0,0,0.001898,0.0,0.003795
6,mark craig,1,0,0,0.001898,0.0,0.003795
7,stuart simek,1,0,0,0.001898,0.0,0.003795
8,michael samp,1,0,0,0.001898,0.0,0.003795
9,john shaeffer,1,0,0,0.001898,0.0,0.003795


In [19]:
#Sort the author by score
author.sort_values("score",inplace=True)

In [20]:
#Tag each author with its rank. The author with highest score is with highest rank
author.reset_index(drop=True,inplace=True)
author['rank'] = 0
for i in range(len(author)):
    author['rank'][i] = i+1

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [21]:
author

,Author,num_of_work,num_of_like,time_with_copa,score_num_of_work,score_num_of_like,score,rank
0,william dillon,1,0,0,0.001898,0.0,0.003795,1
1,mike mccarthy,1,0,0,0.001898,0.0,0.003795,2
2,gordon feingold,1,0,0,0.001898,0.0,0.003795,3
3,scott philiben,1,0,0,0.001898,0.0,0.003795,4
4,stephen upton,1,0,0,0.001898,0.0,0.003795,5
5,chris rose,1,0,0,0.001898,0.0,0.003795,6
6,michael harris,1,0,0,0.001898,0.0,0.003795,7
7,jim lewis,1,0,0,0.001898,0.0,0.003795,8
8,roberto dabed,1,0,0,0.001898,0.0,0.003795,9
9,john evans,1,0,0,0.001898,0.0,0.003795,10


In [22]:
author_rank = author[['Author','rank']]
author_rank.columns = ['Author','author_score']
author_rank #Milestone1:obtain all authors and their ranks as their final score

,Author,author_score
0,william dillon,1
1,mike mccarthy,2
2,gordon feingold,3
3,scott philiben,4
4,stephen upton,5
5,chris rose,6
6,michael harris,7
7,jim lewis,8
8,roberto dabed,9
9,john evans,10


In [23]:
df1 = df
for i in range(len(df)):
    if (type(df['Author'][i])==str):
        df1['Author'][i] = df['Author'][i].lower()
        
result = pd.merge(df1, author_rank, how='left', on=['Author'])

for i in range(len(result)):
    if (np.isnan(result['author_score'][i])):
        result['author_score'][i] = 0

result.head(100) #Milestone2: match all authors in the df with their score.

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


,Unnamed: 0,Article,Author,Comment,Content,Data_source,Date,Like,Most_common_words,Table of Contents,Title,author_score
0,0,NaN,rick beach,0.0,"by RICK BEACH, COPA?? SAFETY CHAIR\nThis unusu...",blog,12/30/2018 23:52,4.0,"['caps', 'pilot', 'accident', 'rocket', 'deplo...",How did the accident unfold?What evidence was ...,"CAPS Deployment Anomaly at Colorado Springs, C...",27.0
1,1,NaN,rick beach,8.0,Every Cirrus aircraft built has a parachute sy...,blog,1/17/2016 0:50,0.0,"['caps', 'low', 'altitude', 'pull', 'attitude'...",How Low?How High?Know your Aircraft: The G5 is...,CAPS: How Low Can You Go? Or How High Do You N...,27.0
2,2,NaN,rick beach,2.0,(This article appears as the safety column in ...,blog,9/13/2017 23:27,1.0,"['cirrus', 'training', 'new', 'transition', 't...",Reach the Unreachable Low time-in-type vs low ...,"""Reach the Unreachable"" safety column in COPA ...",27.0
3,3,NaN,rick beach,0.0,This column will appear in the Nov/Dec 2018 is...,blog,10/15/2018 4:22,12.0,"['runway', 'figure', 'left', 'airplane', 'wing...",NaN,?Â?That Sounds Expensive?Â? -- It Happened t...,27.0
4,4,NaN,rick beach,4.0,A Note from the Guest Editor\nThis June 2017 i...,blog,6/10/2017 18:11,0.0,"['copa', 'training', 'cirrus', 'safety', 'pilo...",A Note from the Guest Editor Success has Many ...,Guest Editor column to Special Safety Issue of...,27.0
5,5,NaN,rick beach,0.0,CaresCritical Incident Response ProgramCIRP i...,blog,8/27/2019 23:15,0.0,"['cirp', 'copa', 'community', 'peer', 'support...",Critical Incident Response for the COPA Commun...,Critical Incident Response for the COPA Community,27.0
6,6,NaN,rick beach,5.0,"For??years,??safety??advocates??have??preached...",blog,1/17/2016 0:51,4.0,"['loss', 'control', 'accidents', 'flight', 'ci...",Cirrus Aircraft Studied Loss of ControlAnalysi...,COPA/Cirrus Aircraft Examine Loss of Control A...,27.0
7,7,NaN,rick beach,0.0,Having participated in several annual meetings...,blog,12/1/2015 18:52,1.0,"['ntsb', 'type', 'investigation', 'investigati...",NaN,Ways that Aviation Type Clubs can prepare and ...,27.0
8,8,NaN,rick beach,2.0,Compare the past six months\r\nwith the same t...,blog,12/8/2012 11:03,0.0,"['caps', 'accidents', 'year', '2012', 'copa', ...",NaN,"In Last Six Months, Half the Fatal Accidents a...",27.0
9,9,NaN,rick beach,5.0,In a recent discussion of the survivable CAPS ...,blog,11/30/2014 18:26,3.0,"['engine', 'fuel', 'oil', 'caps', 'power', 'co...",Recent Fatal Accidents BAD WEATHER FLYINGPILOT...,Turning Fatals into CAPS Saves and Avoiding CA...,27.0


In [24]:
result['datasource_score'] = 0

In [25]:
#For data source, give magazine 3 points, wikis and blogs are all 1 points.
result.loc[result['Data_source']=='magazine','datasource_score']=3
result.loc[result['Data_source']=='wikis','datasource_score']=1
result.loc[result['Data_source']=='blog','datasource_score']=1

In [26]:
result.head(100)

,Unnamed: 0,Article,Author,Comment,Content,Data_source,Date,Like,Most_common_words,Table of Contents,Title,author_score,datasource_score
0,0,NaN,rick beach,0.0,"by RICK BEACH, COPA?? SAFETY CHAIR\nThis unusu...",blog,12/30/2018 23:52,4.0,"['caps', 'pilot', 'accident', 'rocket', 'deplo...",How did the accident unfold?What evidence was ...,"CAPS Deployment Anomaly at Colorado Springs, C...",27.0,1
1,1,NaN,rick beach,8.0,Every Cirrus aircraft built has a parachute sy...,blog,1/17/2016 0:50,0.0,"['caps', 'low', 'altitude', 'pull', 'attitude'...",How Low?How High?Know your Aircraft: The G5 is...,CAPS: How Low Can You Go? Or How High Do You N...,27.0,1
2,2,NaN,rick beach,2.0,(This article appears as the safety column in ...,blog,9/13/2017 23:27,1.0,"['cirrus', 'training', 'new', 'transition', 't...",Reach the Unreachable Low time-in-type vs low ...,"""Reach the Unreachable"" safety column in COPA ...",27.0,1
3,3,NaN,rick beach,0.0,This column will appear in the Nov/Dec 2018 is...,blog,10/15/2018 4:22,12.0,"['runway', 'figure', 'left', 'airplane', 'wing...",NaN,?Â?That Sounds Expensive?Â? -- It Happened t...,27.0,1
4,4,NaN,rick beach,4.0,A Note from the Guest Editor\nThis June 2017 i...,blog,6/10/2017 18:11,0.0,"['copa', 'training', 'cirrus', 'safety', 'pilo...",A Note from the Guest Editor Success has Many ...,Guest Editor column to Special Safety Issue of...,27.0,1
5,5,NaN,rick beach,0.0,CaresCritical Incident Response ProgramCIRP i...,blog,8/27/2019 23:15,0.0,"['cirp', 'copa', 'community', 'peer', 'support...",Critical Incident Response for the COPA Commun...,Critical Incident Response for the COPA Community,27.0,1
6,6,NaN,rick beach,5.0,"For??years,??safety??advocates??have??preached...",blog,1/17/2016 0:51,4.0,"['loss', 'control', 'accidents', 'flight', 'ci...",Cirrus Aircraft Studied Loss of ControlAnalysi...,COPA/Cirrus Aircraft Examine Loss of Control A...,27.0,1
7,7,NaN,rick beach,0.0,Having participated in several annual meetings...,blog,12/1/2015 18:52,1.0,"['ntsb', 'type', 'investigation', 'investigati...",NaN,Ways that Aviation Type Clubs can prepare and ...,27.0,1
8,8,NaN,rick beach,2.0,Compare the past six months\r\nwith the same t...,blog,12/8/2012 11:03,0.0,"['caps', 'accidents', 'year', '2012', 'copa', ...",NaN,"In Last Six Months, Half the Fatal Accidents a...",27.0,1
9,9,NaN,rick beach,5.0,In a recent discussion of the survivable CAPS ...,blog,11/30/2014 18:26,3.0,"['engine', 'fuel', 'oil', 'caps', 'power', 'co...",Recent Fatal Accidents BAD WEATHER FLYINGPILOT...,Turning Fatals into CAPS Saves and Avoiding CA...,27.0,1


In [27]:
#Transform NaN in 'Like' column to 0.
for i in range(len(result)):
    if (np.isnan(result['Like'][i])):
        result['Like'][i] = 0

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [28]:
result.head(100)

,Unnamed: 0,Article,Author,Comment,Content,Data_source,Date,Like,Most_common_words,Table of Contents,Title,author_score,datasource_score
0,0,NaN,rick beach,0.0,"by RICK BEACH, COPA?? SAFETY CHAIR\nThis unusu...",blog,12/30/2018 23:52,4.0,"['caps', 'pilot', 'accident', 'rocket', 'deplo...",How did the accident unfold?What evidence was ...,"CAPS Deployment Anomaly at Colorado Springs, C...",27.0,1
1,1,NaN,rick beach,8.0,Every Cirrus aircraft built has a parachute sy...,blog,1/17/2016 0:50,0.0,"['caps', 'low', 'altitude', 'pull', 'attitude'...",How Low?How High?Know your Aircraft: The G5 is...,CAPS: How Low Can You Go? Or How High Do You N...,27.0,1
2,2,NaN,rick beach,2.0,(This article appears as the safety column in ...,blog,9/13/2017 23:27,1.0,"['cirrus', 'training', 'new', 'transition', 't...",Reach the Unreachable Low time-in-type vs low ...,"""Reach the Unreachable"" safety column in COPA ...",27.0,1
3,3,NaN,rick beach,0.0,This column will appear in the Nov/Dec 2018 is...,blog,10/15/2018 4:22,12.0,"['runway', 'figure', 'left', 'airplane', 'wing...",NaN,?Â?That Sounds Expensive?Â? -- It Happened t...,27.0,1
4,4,NaN,rick beach,4.0,A Note from the Guest Editor\nThis June 2017 i...,blog,6/10/2017 18:11,0.0,"['copa', 'training', 'cirrus', 'safety', 'pilo...",A Note from the Guest Editor Success has Many ...,Guest Editor column to Special Safety Issue of...,27.0,1
5,5,NaN,rick beach,0.0,CaresCritical Incident Response ProgramCIRP i...,blog,8/27/2019 23:15,0.0,"['cirp', 'copa', 'community', 'peer', 'support...",Critical Incident Response for the COPA Commun...,Critical Incident Response for the COPA Community,27.0,1
6,6,NaN,rick beach,5.0,"For??years,??safety??advocates??have??preached...",blog,1/17/2016 0:51,4.0,"['loss', 'control', 'accidents', 'flight', 'ci...",Cirrus Aircraft Studied Loss of ControlAnalysi...,COPA/Cirrus Aircraft Examine Loss of Control A...,27.0,1
7,7,NaN,rick beach,0.0,Having participated in several annual meetings...,blog,12/1/2015 18:52,1.0,"['ntsb', 'type', 'investigation', 'investigati...",NaN,Ways that Aviation Type Clubs can prepare and ...,27.0,1
8,8,NaN,rick beach,2.0,Compare the past six months\r\nwith the same t...,blog,12/8/2012 11:03,0.0,"['caps', 'accidents', 'year', '2012', 'copa', ...",NaN,"In Last Six Months, Half the Fatal Accidents a...",27.0,1
9,9,NaN,rick beach,5.0,In a recent discussion of the survivable CAPS ...,blog,11/30/2014 18:26,3.0,"['engine', 'fuel', 'oil', 'caps', 'power', 'co...",Recent Fatal Accidents BAD WEATHER FLYINGPILOT...,Turning Fatals into CAPS Saves and Avoiding CA...,27.0,1


In [29]:
#Transform NaN in 'Comment' column to 0.
for i in range(len(result)):
    if (np.isnan(result['Comment'][i])):
        result['Comment'][i] = 0

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [30]:
result['like_score'] = result['Like']
result['comment_score'] = result['Comment']

In [31]:
result.head(100)

,Unnamed: 0,Article,Author,Comment,Content,Data_source,Date,Like,Most_common_words,Table of Contents,Title,author_score,datasource_score,like_score,comment_score
0,0,NaN,rick beach,0.0,"by RICK BEACH, COPA?? SAFETY CHAIR\nThis unusu...",blog,12/30/2018 23:52,4.0,"['caps', 'pilot', 'accident', 'rocket', 'deplo...",How did the accident unfold?What evidence was ...,"CAPS Deployment Anomaly at Colorado Springs, C...",27.0,1,4.0,0.0
1,1,NaN,rick beach,8.0,Every Cirrus aircraft built has a parachute sy...,blog,1/17/2016 0:50,0.0,"['caps', 'low', 'altitude', 'pull', 'attitude'...",How Low?How High?Know your Aircraft: The G5 is...,CAPS: How Low Can You Go? Or How High Do You N...,27.0,1,0.0,8.0
2,2,NaN,rick beach,2.0,(This article appears as the safety column in ...,blog,9/13/2017 23:27,1.0,"['cirrus', 'training', 'new', 'transition', 't...",Reach the Unreachable Low time-in-type vs low ...,"""Reach the Unreachable"" safety column in COPA ...",27.0,1,1.0,2.0
3,3,NaN,rick beach,0.0,This column will appear in the Nov/Dec 2018 is...,blog,10/15/2018 4:22,12.0,"['runway', 'figure', 'left', 'airplane', 'wing...",NaN,?Â?That Sounds Expensive?Â? -- It Happened t...,27.0,1,12.0,0.0
4,4,NaN,rick beach,4.0,A Note from the Guest Editor\nThis June 2017 i...,blog,6/10/2017 18:11,0.0,"['copa', 'training', 'cirrus', 'safety', 'pilo...",A Note from the Guest Editor Success has Many ...,Guest Editor column to Special Safety Issue of...,27.0,1,0.0,4.0
5,5,NaN,rick beach,0.0,CaresCritical Incident Response ProgramCIRP i...,blog,8/27/2019 23:15,0.0,"['cirp', 'copa', 'community', 'peer', 'support...",Critical Incident Response for the COPA Commun...,Critical Incident Response for the COPA Community,27.0,1,0.0,0.0
6,6,NaN,rick beach,5.0,"For??years,??safety??advocates??have??preached...",blog,1/17/2016 0:51,4.0,"['loss', 'control', 'accidents', 'flight', 'ci...",Cirrus Aircraft Studied Loss of ControlAnalysi...,COPA/Cirrus Aircraft Examine Loss of Control A...,27.0,1,4.0,5.0
7,7,NaN,rick beach,0.0,Having participated in several annual meetings...,blog,12/1/2015 18:52,1.0,"['ntsb', 'type', 'investigation', 'investigati...",NaN,Ways that Aviation Type Clubs can prepare and ...,27.0,1,1.0,0.0
8,8,NaN,rick beach,2.0,Compare the past six months\r\nwith the same t...,blog,12/8/2012 11:03,0.0,"['caps', 'accidents', 'year', '2012', 'copa', ...",NaN,"In Last Six Months, Half the Fatal Accidents a...",27.0,1,0.0,2.0
9,9,NaN,rick beach,5.0,In a recent discussion of the survivable CAPS ...,blog,11/30/2014 18:26,3.0,"['engine', 'fuel', 'oil', 'caps', 'power', 'co...",Recent Fatal Accidents BAD WEATHER FLYINGPILOT...,Turning Fatals into CAPS Saves and Avoiding CA...,27.0,1,3.0,5.0


The result here is ready for user input. Once user input words match with Most_common_words, calculate the match score, then normalize and add up with author_score, datasource_score, like_score, and comment_score to get the final score for each row. Then rank the result according to the final total score to get the optimized putput.